In [63]:
import pandas as pd
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler 
from sklearn.neural_network import MLPClassifier  
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings("ignore")

# 在開始之前，你或許可以從這裡了解神經網路：https://okai.brown.edu/zh/index.html

# 核心：sklearn 套件包的 MLPClassifier 機器學習分類工具
# 主要參考來源：https://stackabuse.com/introduction-to-neural-networks-with-scikit-learn/
# 其他參考資料如下：
# https://datascience.stackexchange.com/questions/16819/back-propagation-and-structure-of-a-neural-network-in-scikit-neuralnetwork
# https://www.datacamp.com/community/tutorials/scikit-learn-python

In [64]:
#====================================================================================================
# 準備資料
#====================================================================================================

# 先定義資料每一列的名稱
# names = ['Att_Avg', 'Att_Std', 'Eye_Time', 'Eye_Freq', 'Eye_Avg', 'Likert']

# 從 BPN.csv 中讀取資料，名稱使用上面定義的名稱，捨棄資料中的標頭（第一行）
# pd.read_csv 讀取 CSV 檔案
# names 列名列表 = 我們定義的 names 名稱列表
# header=0 表示第一行數據而不是文件的第一行
data = pd.read_csv('Data/BPN_Data/Eye.csv', header = 0)

# 印出來看看有沒有毛病
# head() 顯示資料
# head(5) 顯示資料前 5 筆
print(data.head()) 

# 將前 3 列當作輸入資料，並將第 4 列 Situation 作為目標輸出資料
X = data.iloc[:, 0:-1]
y = data.iloc[:, -1]  

# 一樣印出來看看有沒有出事
print("="*90)
print(X.head())
print("="*90)
print(y.head())


    58  1  58.00  1.1
0   29  2   14.0    0
1   44  3   14.0    1
2   18  2    9.0    0
3  101  3   33.0    1
4   25  5    5.0    0
    58  1  58.00
0   29  2   14.0
1   44  3   14.0
2   18  2    9.0
3  101  3   33.0
4   25  5    5.0
0    0
1    1
2    0
3    1
4    0
Name: 1.1, dtype: int64


In [65]:
def run(roundi, X_train, X_test, y_train, y_test):
    # 建立神經網路
    # 輸入層會由輸入的參數數量自動決定
    # hidden_layer_sizes = 隱藏層的大小，(10, 10, 10) 的情況就是 3 層各 10 個節點
    # 隱藏層的數量最好嘗試一下不同組合的效果再決定
    # max_iter = 希望迭代的次數，一次會執行前饋和反向傳播
    # 也就是說 1000 的話會跑 1000 次前饋和 1000 反饋
    # 默認使用的激活函數是 Relu，可參照：https://mropengate.blogspot.com/2017/02/deep-learning-role-of-activation.html
    # 默認使用的優化算法是 Adam，可參照：https://juejin.im/entry/5983115f6fb9a03c50227fd4
    mlp = MLPClassifier(hidden_layer_sizes=(6), max_iter=1000) 
    # best now: 48, 126, 48, 3000 := 85.14
    # 64, 64, 64, 5000 := 75.28
    mlp.fit(X_train, y_train.values.ravel()) # 就，開始做事

    # 到這邊完成訓練資料

    # 開始使用測試集來預測
    predictions = mlp.predict(X_test) 

    # 印出矩陣和預測結果
    matrix = confusion_matrix(y_test,predictions)  
    result = classification_report(y_test,predictions)
    
    with open('Result_Eye/' + str(roundi) + '.txt', 'w') as f:
        f.writelines(str(matrix))
        f.writelines('\n')
        f.writelines(str(result))
    
    # 主要是 precision = 準確率
    # 至於全部的指標意義，如召回率、微平均等等，請參照：https://zhuanlan.zhihu.com/p/30953081

    acc = sklearn.metrics.accuracy_score(y_test, predictions)
    return acc

skf = StratifiedKFold(n_splits = 10 , shuffle = True)

acc_list = []
i = 0
# 開始切分，這邊會給 測試集 和 訓練集 的標籤，再按照標籤做劃分
for X_index, tX_index in skf.split(X , y):

    X_train = X.iloc[X_index]
    X_test = X.iloc[tX_index]
    
    y_train = y.iloc[X_index]
    y_test = y.iloc[tX_index]

    for j in range(10):
        # 去均值和歸一化的工具
        # 詳細的內容可以參考：https://blog.csdn.net/maqunfi/article/details/82252480
        scaler = StandardScaler()  

        # 將訓練集的特徵放大，讓訓練效果好一點
        scaler.fit(X_train)

        # 對訓練集和測試集都做優化
        X_train = scaler.transform(X_train)  
        X_test = scaler.transform(X_test)

        acc = run(i+1, X_train, X_test, y_train, y_test)
        print("測試集 {:>2d} 號 第 {:>2d} 輪的正確率為： {:.4%}".format(i+1, j+1, acc))
        acc_list.append(acc)
    i += 1
    print('='*50)
    
acc_sum = 0.0
for acc in acc_list:
    acc_sum += acc
    
acc_avg = acc_sum / len(acc_list)
print("總平均 正確率為：{:.4%}".format(acc_avg))

測試集  1 號 第  1 輪的正確率為： 71.4286%
測試集  1 號 第  2 輪的正確率為： 85.7143%
測試集  1 號 第  3 輪的正確率為： 85.7143%
測試集  1 號 第  4 輪的正確率為： 85.7143%
測試集  1 號 第  5 輪的正確率為： 85.7143%
測試集  1 號 第  6 輪的正確率為： 85.7143%
測試集  1 號 第  7 輪的正確率為： 71.4286%
測試集  1 號 第  8 輪的正確率為： 100.0000%
測試集  1 號 第  9 輪的正確率為： 85.7143%
測試集  1 號 第 10 輪的正確率為： 71.4286%
測試集  2 號 第  1 輪的正確率為： 85.7143%
測試集  2 號 第  2 輪的正確率為： 85.7143%
測試集  2 號 第  3 輪的正確率為： 71.4286%
測試集  2 號 第  4 輪的正確率為： 85.7143%
測試集  2 號 第  5 輪的正確率為： 85.7143%
測試集  2 號 第  6 輪的正確率為： 85.7143%
測試集  2 號 第  7 輪的正確率為： 85.7143%
測試集  2 號 第  8 輪的正確率為： 85.7143%
測試集  2 號 第  9 輪的正確率為： 85.7143%
測試集  2 號 第 10 輪的正確率為： 85.7143%
測試集  3 號 第  1 輪的正確率為： 85.7143%
測試集  3 號 第  2 輪的正確率為： 85.7143%
測試集  3 號 第  3 輪的正確率為： 71.4286%
測試集  3 號 第  4 輪的正確率為： 71.4286%
測試集  3 號 第  5 輪的正確率為： 85.7143%
測試集  3 號 第  6 輪的正確率為： 85.7143%
測試集  3 號 第  7 輪的正確率為： 71.4286%
測試集  3 號 第  8 輪的正確率為： 57.1429%
測試集  3 號 第  9 輪的正確率為： 85.7143%
測試集  3 號 第 10 輪的正確率為： 85.7143%
測試集  4 號 第  1 輪的正確率為： 85.7143%
測試集  4 號 第  2 輪的正確率為： 85.7143%
測試集  4 